In [ ]:
import os
import requests
import pandas as pd
from dotenv import load_dotenv
from datetime import datetime, timedelta, timezone

# Load environment variables
load_dotenv()
API_KEY = os.getenv("WEATHERAPI_KEY")  # Ensure correct variable name
BASE_URL = "http://api.weatherapi.com/v1/history.json"

def get_last_24h_weather(city: str):
    """
    Fetch the last 24 hours of hourly weather data for a given city.

    Args:
        city (str): City name (e.g., "London").

    Returns:
        pd.DataFrame: Weather data for the last 24 hours.
    """
    # Fix: Use timezone-aware datetime
    yesterday = (datetime.now(timezone.utc) - timedelta(days=1)).strftime('%Y-%m-%d')
    
    params = {
        "key": API_KEY,
        "q": city,
        "dt": yesterday,
    }

    response = requests.get(BASE_URL, params=params)

    if response.status_code == 200:
        data = response.json()
        hourly_data = data["forecast"]["forecastday"][0]["hour"]
        
        # Convert to DataFrame
        df = pd.DataFrame(hourly_data)
        df["time"] = pd.to_datetime(df["time"])
        
        # Select relevant columns
        df = df[["time", "temp_c", "humidity", "condition"]]
        df["weather_description"] = df["condition"].apply(lambda x: x["text"])
        df.drop(columns=["condition"], inplace=True)

        return df

    else:
        print(f"Error fetching data: {response.status_code} - {response.text}")
        return pd.DataFrame()

# Test API Key
if not API_KEY:
    print("Error: API key not loaded. Check .env file.")
else:
    print("API key loaded successfully.")

# Get weather for London
df_weather = get_last_24h_weather("London")

# Display DataFrame
print(df_weather.head(10))